In [9]:
from agents import Agent, Runner, trace, handoff
from IPython.display import Markdown, display
from pydantic import BaseModel, Field
from dotenv import load_dotenv
load_dotenv(override=True)


True

In [10]:
class Idea(BaseModel):
    title: str = Field(description="The title of the idea")
    description: str = Field(description="A detailed description of the idea, in Markdown")

    def print_summary(self):
        print(f"Title: {self.title}")
        display(Markdown(self.description))

In [11]:
class Evaluation(BaseModel):
    idea: Idea = Field(description="The idea that you are evaluating")

    uniqueness_feedback: str = Field(description="Your view on how unique the idea is")
    uniqueness_score: int = Field(description="How unique the idea is, on a scale of 1 to 10, where 1 is mundane and 10 is groundbreaking")

    commercial_feedback: str = Field(description="Your view on the commercial potential of the idea")
    commercial_score: int = Field(description="How high the commercial potential of the idea is from 1 to 10, where 1 is barely profitable and 10 is a billion dollar idea")

    autonomy_feedback: str = Field(description="Your commentary on to what extent the idea truly benefits from the autonomous nature of AI Agents")
    autonomy_score: int = Field(description="How deeply the idea involves autonomous agents from 1 to 10, where 1 is that it plays no meaningful role, and 10 is that it is pivotal")

    def is_brilliant(self):
        return self.uniqueness_score > 8 and self.commercial_score > 8 and self.autonomy_score > 8

    def print_summary(self):
        print(f"Idea: {self.idea.title}")
        print(f"Uniqueness: {self.uniqueness_score}/10: ({self.uniqueness_feedback})")
        print(f"Commercial: {self.commercial_score}/10: ({self.commercial_feedback})")
        print(f"Autonomy: {self.autonomy_score}/10: ({self.autonomy_feedback})")

In [ ]:
# 1. Improved Agent Instructions
# The Entrepreneur must be explicitly told to IMPROVE the previous idea.
# The Evaluator must be explicitly told to pass the IDEA (inside the Evaluation) back.

entrepreneur_instructions = "You generate and refine novel, exciting business ideas for Autonomous AI Agents. Your primary goal is to produce an idea that scores 9 or 10 in all three categories (Uniqueness, Commercial Potential, Autonomy)."

entrepreneur_input = """
Propose the initial compelling business idea for applying Autonomous AI Agents to any industry you choose.
The idea should be unique, high potential commercial impact, and make true use of the autonomous capabilities of AI Agents.
"""
# explicitly tell the Entrepreneur to refine based on the feedback
entrepreneur_input += """
If you receive an Evaluation, you MUST analyze the feedback and scores. You must then generate an improved version of the Idea (contained within the Evaluation) and handoff the improved idea to the Evaluator for a new assessment.
You MUST handoff your idea or improved idea to the Evaluator by using the tool Handoff_to_Evaluator.
"""

evaluator_instructions = """
You evaluate novel, exciting business ideas for Autonomous AI Agents.
Evaluate the business idea that you are given; assess it for uniqueness, potential commercial impact, and use of Autonomous AI Agents.
"""
# explicityly tell the Evaluator to halt on success
evaluator_instructions += """
CRITICAL LOGIC:
1. If the idea scores 9/10 or 10/10 in ALL 3 categories, then output the Evaluation as your final response to STOP the process.
2. If ANY score is 8/10 or lower, you must handoff the Idea and your feedback back to the Entrepreneur to improve it.
You MUST use the tool Handoff_to_Entrepreneur, passing the entire Evaluation, which includes the Idea and your feedback.
"""

In [14]:
# 2. Agent Setup (Unchanged - using more capable model is recommended, but not essential for structure)
# entrepreneur = Agent(name="Entrepreneur", model="gpt-4o", instructions=entrepreneur_instructions, output_type=Idea)
# evaluator = Agent(name="Evaluator", model="gpt-4o", instructions=evaluator_instructions, output_type=Evaluation)
entrepreneur = Agent(name="Entrepreneur", model="gpt-4.1-mini", instructions=entrepreneur_instructions, output_type=Idea)
evaluator = Agent(name="Evaluator", model="gpt-4.1-mini", instructions=evaluator_instructions, output_type=Evaluation)

In [17]:
def on_handoff(ctx, input):
    # Try using 'ctx.agent' or 'ctx.current_agent' instead of 'ctx.caller_agent'
    # 'ctx.agent' is the most common convention for the active agent in a hook.
    try:
        caller_name = ctx.agent.name
    except AttributeError:
        # Fallback in case ctx.agent also doesn't exist, to prevent crashing
        caller_name = "Unknown Agent"

    print(f"Handoff called: {caller_name} passed {type(input).__name__}")
# 3. Handoff Definitions (Unchanged, but using the improved on_handoff)

entrepreneur.handoffs = [handoff(
    agent=evaluator,
    input_type=Idea,
    on_handoff=on_handoff,
    tool_name_override="Handoff_to_Evaluator",
    tool_description_override="Use this tool to handoff your idea to the Evaluator to evaluate. You must always call this tool."
)]

evaluator.handoffs = [handoff(
    agent=entrepreneur,
    input_type=Evaluation, # The Evaluation object carries the Idea and the feedback
    on_handoff=on_handoff,
    tool_name_override="Handoff_to_Entrepreneur",
    tool_description_override="Use this tool to handoff your feedback to the Entrepreneur to improve the idea. You should call this tool if your feedback is less than 9/10 in any category."
)]


In [18]:
with trace("Entrepreneur Approach 2"):
    result = await Runner.run(entrepreneur, entrepreneur_input, max_turns=10)

Handoff called: Unknown Agent passed Idea
Handoff called: Unknown Agent passed Evaluation
Handoff called: Unknown Agent passed Idea


In [19]:
if result and result.final_output:
    result.final_output.print_summary()


Idea: AI-Driven Autonomous Mental Wellness Ecosystem
Uniqueness: 9/10: (This enhanced idea is highly unique due to its integration of fully autonomous crisis management and a multi-agent collaborative ecosystem for comprehensive mental wellness support, which is not common in current market offerings.)
Commercial: 9/10: (The added crisis management and predictive capabilities significantly increase commercial viability by addressing critical gaps in mental health support, appealing to both users and healthcare providers, and addressing compliance through blockchain data privacy.)
Autonomy: 9/10: (The use of autonomous AI agents to independently manage crisis situations, coordinate therapy, and predict mental health episodes exemplifies deep and crucial use of AI autonomy, making the system robust and highly autonomous.)


In [ ]:
"""
Parameter,                  Value,                          Purpose
agent                       evaluator                       Specifies the target agent that will receive control next.
input_type                  Idea                            Defines the Pydantic object that the Entrepreneur must generate and pass to the Evaluator when using this tool.
on_handoff                  on_handoff                      Registers a callback function to be executed right after the Entrepreneur calls the tool, but before the Evaluator starts its turn. This is your logging hook."
tool_name_override          Handoff_to_Evaluator            This is the internal tool name that the framework will use to communicate the tool's existence to the LLM. The agent's prompt will be dynamically updated to include this tool.
tool_description_override   Use this tool                   This is the instruction provided to the LLM (as part of its system prompt/tools list) on how and when to use this handoff tool. The instruction here strongly encourages the agent to always use it.
"""